<a href="https://colab.research.google.com/github/roochathakkar05/PhishingDetection/blob/main/FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Modules

In [ ]:
!pip install tldextract
!pip uninstall whois 
!pip install python-whois
import regex    
from tldextract import extract
import ssl
import socket
from bs4 import BeautifulSoup
import urllib, bs4, re
import urllib.request
import googlesearch
import whois
from datetime import datetime, timezone
import time
import pandas as pd
import numpy as np
import re
import os.path
import requests
from urllib.parse import urlparse,urlencode
import ipaddress
import re

Raw Data from csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
urldataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/urldatamal.csv',skiprows=range(1,8006),nrows=1005)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
print(urldataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     1005 non-null   object
 1   label   1005 non-null   object
 2   result  1005 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 23.7+ KB
None


In [ ]:
urldataset.head

<bound method NDFrame.head of                                                     url      label  result
0           http://healthybloodpressure.info/2uOioq.php  malicious       1
1             http://scantanzania.com/bin/img/make.html  malicious       1
2                                    http://technix.it/  malicious       1
3                              http://reifen-simon.com/  malicious       1
4                                    http://szinhaz.hu/  malicious       1
...                                                 ...        ...     ...
1000   http://pageblockingfb.at.ua/incorrect_email.html  malicious       1
1001     http://systemautomatically.co.nf/manager/a.php  malicious       1
1002      http://pageblockingfb.at.ua/confirmation.html  malicious       1
1003  http://encorepaintpros.com/wp-admin/standardza...  malicious       1
1004  http://sign-in.paypal-update-security.webapps....  malicious       1

[1005 rows x 3 columns]>

Feature engineering

In [ ]:
def have_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        return -1
    else:
        return 1
urldataset['have_ip_address'] = urldataset['url'].apply(lambda i: have_ip_address(i))


In [ ]:
def url_length(url):
    if len(url)<54:
        return 1
    else:
        return -1
urldataset['url_length'] = urldataset['url'].apply(lambda i: url_length(i))

In [ ]:
def url_shortener(url):
    match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)

    if match:
        return -1
    else:
        return 1
urldataset['url_shortener'] = urldataset['url'].apply(lambda i: url_shortener(i))

In [ ]:
def have_atrate_symbol(url):
    match = re.search('@',url)

    if match:
        return -1
    else:
        return 1
urldataset['have_atrate_symbol'] = urldataset['url'].apply(lambda i: have_atrate_symbol(i))        

In [ ]:
def have_subdomain(url):
    if(have_ip_address(url)==-1):
        match = re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5]))|(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)
        pos = match.end(0)
        url = url[pos:]
    list = [x.start(0) for x in re.finditer('\.',url)]
    if len(list)<=3:
        return 1
    else:
        return -1
urldataset['have_subdomain'] = urldataset['url'].apply(lambda i: have_subdomain(i))


In [ ]:
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return -1
    else:
      return 1
  else:
    return 1
urldataset['redirection'] = urldataset['url'].apply(lambda i: redirection(i))

In [ ]:
def httpDomain(url):
  if 'https' in url:
    return 1
  else:
    return -1
urldataset['httpDomain'] = urldataset['url'].apply(lambda i: httpDomain(i))

In [ ]:
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return -1            # phishing
    else:
        return 1            # legitimate
urldataset['prefixSuffix'] = urldataset['url'].apply(lambda i: prefixSuffix(i))

In [ ]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth
urldataset['getDepth'] = urldataset['url'].apply(lambda i: getDepth(i))

In [ ]:
def domainAge(url):
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
    print(domain_name)
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date
    if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
      try:
        creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
      except:
        return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 1
    else:
      ageofdomain = abs((expiration_date - creation_date).days)
      if ((ageofdomain/30) < 6):
        age = 1
      else:
        age = 0
      return age
  except:
    dns = 1

urldataset['domainAge'] = urldataset['url'].apply(lambda i: domainAge(i))


Streaming output truncated to the last 5000 lines.
  "status": [
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientTransferProhibited - http://www.icann.org/epp#clientTransferProhibited"
  ],
  "emails": "abuse@1api.net",
  "dnssec": "unsigned",
  "name": "REDACTED FOR PRIVACY",
  "org": "REDACTED FOR PRIVACY",
  "address": "REDACTED FOR PRIVACY",
  "city": "REDACTED FOR PRIVACY",
  "state": "Tehran",
  "zipcode": "REDACTED FOR PRIVACY",
  "country": "IR"
}
{
  "domain_name": [
    "80PORT.NET",
    "80port.net"
  ],
  "registrar": "INAMES CO., LTD.",
  "whois_server": "whois.inames.co.kr",
  "referral_url": null,
  "updated_date": [
    "2020-07-01 06:08:54",
    "2020-07-01 06:08:53"
  ],
  "creation_date": "2001-08-01 17:36:23",
  "expiration_date": "2021-08-01 17:36:23",
  "name_servers": [
    "NS.80PORT.COM",
    "NS1.80PORT.COM",
    "ns.80port.com",
    "ns1.80port.com"
  ],
  "status": [
    "clientTransferProhibited https://icann.org/ep

In [ ]:
urldataset.head()

,url,label,result,have_ip_address,url_length,url_shortener,have_atrate_symbol,have_subdomain,redirection,httpDomain,prefixSuffix,getDepth,domainAge
0,http://healthybloodpressure.info/2uOioq.php,malicious,1,1,1,1,1,1,1,-1,1,1,1.0
1,http://scantanzania.com/bin/img/make.html,malicious,1,1,1,1,1,1,1,-1,1,3,0.0
2,http://technix.it/,malicious,1,1,1,1,1,1,1,-1,1,0,0.0
3,http://reifen-simon.com/,malicious,1,1,1,1,1,1,1,-1,-1,0,0.0
4,http://szinhaz.hu/,malicious,1,1,1,1,1,1,1,-1,1,0,1.0


In [ ]:
urldataset.tail()

,url,label,result,have_ip_address,url_length,url_shortener,have_atrate_symbol,have_subdomain,redirection,httpDomain,prefixSuffix,getDepth,domainAge
1000,http://pageblockingfb.at.ua/incorrect_email.html,malicious,1,1,1,1,1,1,1,-1,1,1,1.0
1001,http://systemautomatically.co.nf/manager/a.php,malicious,1,1,1,1,1,1,1,-1,1,2,1.0
1002,http://pageblockingfb.at.ua/confirmation.html,malicious,1,1,1,1,1,1,1,-1,1,1,1.0
1003,http://encorepaintpros.com/wp-admin/standardza...,malicious,1,1,-1,1,1,1,1,-1,1,3,1.0
1004,http://sign-in.paypal-update-security.webapps....,malicious,1,1,-1,1,1,-1,1,-1,-1,0,NaN


In [ ]:
urldataset.to_csv('/content/drive/My Drive/Colab Notebooks/Urldata1.csv')

In [ ]:
urldataset1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata1.csv')
urldataset2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata2.csv')
urldataset3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata3.csv')
urldataset4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata4.csv')
urldataset5 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata5.csv')
urldataset6 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata6.csv')
urldataset7 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata7.csv')
urldataset8 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata8.csv')
urldataset9 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata9.csv')
urldataset10 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata10.csv')
urldataset11 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata11.csv')
urldataset12 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Urldata12.csv')

In [ ]:
frames = [urldataset1,urldataset2,urldataset3,urldataset4,urldataset5,urldataset6,urldataset7,urldataset8,urldataset9,urldataset10,urldataset11,urldataset12]

In [ ]:
result = pd.concat(frames)

In [ ]:
result.to_csv('/content/drive/My Drive/Colab Notebooks/urldata.csv')